# Homework Assignment 5: Model Evaluation
As in the previous assignments, in this homework assignment you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).


This assignment will utilize a copy of the extracted feature dataset we have been working with. The dataset has been processed by performing outlier clipping, z-score and range scaling, and forward feature selection to select 20 features. We are now going to utilize more than one partition worth of data, so for the z-score and range scaling, the mean, standard deviation, minimum, and maximum were calculated using data from both partitions so that a global scaling can be performed on each partition. 

---

## Step 1: Downloading the Data

This assignment will continue to only use [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB) and will add the use of [Partition 2](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/TCRPUD) as a testing set. 

---
#### Homework 1 & 2

Recall, that in Homework 1, we started to construct the analytics base table for our [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM). In that assignment, we read the data from the two subdirectories, __FL__ and __NF__, of the __partition1__ direcotry. These two subdirectories represented the two classes of our target feature in the solar flare prediction problem we are attempting to solve this semester. We then processed these samples of multivariate time series to construct descriptive features for each sample, and then placed them into our analytics base table.

Then, in Homework 2, you utilized a set of extracted descriptive features much like what you were asked to construct in Homework 1. However, this dataset contained many more extracted features than you were asked to compute for Homework 1 (>800). So, we needed to explore the data to find data quality issues and identify ways to address any we found. Below are the links to the full extracted feature for all of partitions 1 and 2, and a toy representative dataset of partition 1 that was used as input to Homework 2.

- [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv)
- [Full Partition 2 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition2ExtractedFeatures.csv)
- [Toy Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_partition1ExtractedFeatures.csv)

---

#### Homework 3

Then, in Homework 3, you were asked to perform additional data preprocessing on data that would have been produced from Homework 2. These preprocessing steps included finding features with large ranges and features with a large number of outliers. You were asked to clip some of the outliers for the features you found and were also asked to perform a few different types of scaling, such as decimal and z-score. The links to those files are below.  

- [Full Cleaned Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/cleaned_partition1ExtractedFeatures.csv)
- [Toy Cleaned Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_cleaned_partition1ExtractedFeatures.csv)
- [Data Quality Table for Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/data_quality_table.csv)

---

#### Homework 4

I then did much more of this preprocessing for you to produce data for Homework 4, inclding the clipping of outliers, and performing z-score and range normalization. I constructed both a full normalized and a toy normalized data file for use in that assignment found below.

- [Full Normalized Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/normalized_partition1ExtractedFeatures.csv)
- [Toy Normalized Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_normalized_partition1ExtractedFeatures.csv)

You were then asked to remove columns that had too many NaN or Inf values in them and replace the remaining NaN and Inf values with the median of the feature the values happend to fall in. Then you were asked to perform various types of feature selection on the features and find a set that we might want to use for classification later.

---

#### Now

For this assignment, I have again performed clipping of outliers, and performing z-score and range normalization, but this time the calculations were based upon the values of partitions one and two.  I also performed the same Nan and Inf processing you were asked to do in Homework 4. I then performed the last feature selection method you were asked to perform using [scikit-learn Sequential Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#sequential-feature-selection) with [scikit-learn LassoLarsCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLarsCV.html#sklearn.linear_model.LassoLarsCV) as the estimator. I then constructed a subset of 20 features for both partition 1 and 2, which will be the input data for this assignment. Below you will find the links to these two files.

- [Partition 1 selected feature dataset](http://dmlab.cs.gsu.edu/solar/data/normalized_partition1SelectedFeatures.csv)
- [Partition 2 selected feature dataset](http://dmlab.cs.gsu.edu/solar/data/normalized_partition2SelectedFeatures.csv)

Now that you have the two files of selected features csv files, you should load each into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method. 

---

### Evaluation Metric

For each of the models we evaluate in this assignmnet, you will calculate the True Skill Statistic score using the test data from Partition 2 to determine which model performs the best for classifying the positive flaring class.

    True skill statistic (TSS) = TPR + TNR - 1 = TPR - (1-TNR) = TPR - FPR

Where:

    True positive rate (TPR) = TP/(TP+FN) Also known as recall or sensitivity
    True negative rate (TNR) = TN/(TN+FP) Also known as specificity or selectivity
    False positive rate (FPR) = FP/(FP+TN) = (1-TNR) Also known as fall-out or false alarm ratio


**Recall**

    True positive (TP)
    True negative (TN)
    False positive (FP)
    False negative (FN)
    
See [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) for more information.

Below is a function implemented to provide your score for each model.

In [ ]:
import os
import itertools
import pandas as pd
from pandas import DataFrame 
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
def calc_tss(y_true=None, y_predict=None):
    """
    Calculates the true skill score for binary classification based on the output of the confusion
    table function
    """
    scores = confusion_matrix(y_true, y_predict).ravel()
    TN, FP, FN, TP = scores
    print('TN={0}\tFP={1}\tFN={2}\tTP={3}'.format(TN, FP, FN, TP))
    tp_rate = TP / float(TP + FN) if TP > 0 else 0  
    fp_rate = FP / float(FP + TN) if FP > 0 else 0
    
    return tp_rate - fp_rate

In [ ]:
data_dir = '/data/FDS'
data_file = "normalized_partition1SelectedFeatures.csv"
data_file2 = "normalized_partition2SelectedFeatures.csv"

In [ ]:
abt = pd.read_csv(os.path.join(data_dir, data_file))
abt2 = pd.read_csv(os.path.join(data_dir, data_file2))

---
### Q1 (10 points)

Now that you have the normalized and selected the top 20 features, you need to prepare a dataset for binary classification. So, write a function that constructs and returns a copy of the analytics base table that is passed in. The function should also change the labels for each entry to be either a 1 or a 0 based upon what the original label was. The 1 will be for `flaring` positive samples, and the 0 will be for `non-flaring` negative samples. You should then apply it to both partition 1 and partition 2 to get a copy to use for training and testing of models in later questions. 

**Labels:** In the partition data, those labeled as M or X will be the positive flaring class, and those labled as C, B, or NF will be the negative flaring class. You may wish to use the use the [replace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html) function of the coppied DataFrame to change the values of the label column.

Below is a function stub for you to complete:

---

In [ ]:
def copy_and_convert_to_binary(data:DataFrame)->DataFrame:
    data.replace({'lab':{'M': 1, 'X': 1, 'C': 0, 'B': 0, 'NF': 0}}, inplace=True)
    return data

In [ ]:
abt_binary_cpy = copy_and_convert_to_binary(abt)
abt2_binary_cpy = copy_and_convert_to_binary(abt2)

---
### Q2 (10 points)

With your binary classification dataset constructed, now it's time to start training and testing some models. We will start with the simple [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), and try several different settings to see how/if using different settings will improve our score. So, using your copy of partition 1 that has had its `lab` column converted to a binary label, train 4 different instances with the following settings. **(see documentation to know what these are)**

|Model Number| n_neighbors | p |
|------------|-------------|---|
|1|3|1|
|2|3|2|
|3|5|1|
|4|5|2|


Once you have done that, test each of your models using your binary classification copy of partition 2, then calculate and print the TSS score for each result. **NOTE: The model does take a little while to evaluate.**

---

In [ ]:
info_dict = {
    1: [3,1],
    2: [3,2],
    3: [5,1],
    4: [5,2]
}
for key in info_dict.keys():
    neighbor = KNeighborsClassifier(n_neighbors=info_dict.get(key)[0], p=info_dict.get(key)[1])
    neighbor.fit(abt_binary_cpy.drop('lab', axis=1),abt_binary_cpy['lab'])
    score = calc_tss(abt2_binary_cpy['lab'], neighbor.predict(abt2_binary_cpy.drop('lab', axis=1)))
    print(f'Model {key} TSS score: {score}')

---
### Q3 (10 points)

After evaluating the various results from Q2, you will notice that the results are not all that great with greater than 1000 false negatives for each of our settings tried. But, what can be done to improve our results? If you read the documentation for the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), which you certainly should have, you will see that we were only using the `MinkowskiDistance` metric with different values of `p`. If you look into the [DistanceMetric](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html#sklearn.neighbors.DistanceMetric) documentation for the neighbors classifiers, you will see there are several others available to use.

So, for this question, train and evaluate two more instances of [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), but this time using the `ChebyshevDistance` metric instead of the `MinkowskiDistance` metric.  For these two models you will only be changing the number neighbors to 3 and 5, as the values of `p` are not used for the `ChebyshevDistance` metric. 

---

In [ ]:
neighbor5 = KNeighborsClassifier(n_neighbors=3, metric='chebyshev')
neighbor5.fit(abt_binary_cpy.drop('lab', axis=1),abt_binary_cpy['lab'])
score5 = calc_tss(abt2_binary_cpy['lab'], neighbor5.predict(abt2_binary_cpy.drop('lab', axis=1)))
print(f'chebyshev 1 TSS score: {score5}')


neighbor6 = KNeighborsClassifier(n_neighbors=5, metric='chebyshev')
neighbor6.fit(abt_binary_cpy.drop('lab', axis=1),abt_binary_cpy['lab'])
score6 = calc_tss(abt2_binary_cpy['lab'], neighbor6.predict(abt2_binary_cpy.drop('lab', axis=1)))
print(f'chebyshev 2 TSS score: {score6}')

---
### Q4 (10 points)

After evaluating the results from Q3, you will see that the results are even worse than those we found for Q2. This leads to the thought that maybe the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) is just not a good fit for the problem we are applying it to. So, let's move on to another classifier for this problem. 

In this question, you will utilize the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), and try several different settings to see how/if using different settings will improve our score. So, using your copy of partition 1 that has had its `lab` column converted to a binary label, train 8 different instances with the following settings. **(see documentation to know what these are)**

|Model Number| criterion | max_depth | splitter |
|------------|---------|-------------|---|
|1|gini|5|best|
|2|gini|5|random|
|3|gini|None|best|
|4|gini|None|random|
|5|entropy|5|best|
|6|entropy|5|random|
|7|entropy|None|best|
|8|entropy|None|random|



Once you have done that, test each of your models using your binary classification copy of partition 2, then calculate and print the TSS score for each result.

---

In [ ]:
info_dict = {
    1:	['gini', 5,	'best'],
    2:	['gini', 5, 'random'],
    3:	['gini', None, 'best'],
    4:	['gini', None, 'random'],
    5:	['entropy', 5, 'best'],
    6:	['entropy', 5, 'random'],
    7:	['entropy', None, 'best'],
    8:	['entropy', None, 'random']
}
for key in info_dict.keys():
    clf = DecisionTreeClassifier(criterion=info_dict.get(key)[0], splitter=info_dict.get(key)[2], max_depth=info_dict.get(key)[1])
    clf.fit(abt_binary_cpy.drop('lab', axis=1),abt_binary_cpy['lab'])
    score = calc_tss(abt2_binary_cpy['lab'], clf.predict(abt2_binary_cpy.drop('lab', axis=1)))
    print(f'Model {key} TTS score: {score}')

---
### Q5 (10 points)

After evaluating results from Q4, you will see that the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) was able to accomplish about 3X improvement over the best resutls we found for the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).  This is indeed great, but can we do better than this if we use yet another classifier? Let's move on to yet another and find out.

For this question you will be utilizing the [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) classifier. We won't be changing any of the default settings, just train 1 model with your copy of partition 1 that has had its `lab` column converted to a binary label. You will then test the model using your binary classification copy of partition 2, then calculate and print the TSS score for the result.

---

In [ ]:
clf = GaussianNB()
clf.fit(abt_binary_cpy.drop('lab', axis=1),abt_binary_cpy['lab'])
score5 = calc_tss(abt2_binary_cpy['lab'], clf.predict(abt2_binary_cpy.drop('lab', axis=1)))
print(f'GaussianNB TSS score: {score5}')

---
### Q6 (10 points)

If you recall from a lecture some time back, it was shown that another way of improving the results of classification is to perform some form of sampling to balance the number of samples there are for the various classes. The reason why this works for specific classifiers, and methods for do the sampling, are as numerious and we don't have enough time to cover them in this cours.  However, it is still beneficial to know this works and that it is something that you should be considering when you are training models.  

So, for this quest, we will implement a very naive method for sampling so we can use the results for training our models again.  Below you will find a function stub, complete the function and have it return a copy of the input dataframe where each class (except for the smallest one) have been undersampled to match the size of the smallest class in the dataset. In this function you should assume the `lab` column is the class label.

To do this you may want to use the [groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) function of the DataFrame to get groups of rows from your DataFrame.  You may also wish to use the [sample](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) function to select a number of rows from a group. You can also use the [apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) method to process each group from your grouped rows. These are just hints, you can solve the problem how you see fit.

Once this function is complete, apply it to the original analytics base table for partition 1 (the one with all the NF, C, .., X labels). Then apply your function that converts the multi-class problem to a binary problem to the results so we can use this new undersampled data for the next several questions.

---

In [ ]:
def perform_under_sample(data:DataFrame)->DataFrame:
    data = data.groupby(['lab'], group_keys=False)
    return pd.DataFrame(data.apply(lambda x: x.sample(data.size().min()))).reset_index(drop=True)

In [ ]:
sampled_abt = perform_under_sample(abt)
abt_binary_cpy = copy_and_convert_to_binary(sampled_abt)

---
### Q7

For this question repeat what you did for Q2, but with your balanced binary classification dataset constructed in Q6, uese the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), and try several different settings to see how/if using different settings will improve our score. 

So, train 4 different instances with the following settings. **(see documentation to know what these are)**

|Model Number| n_neighbors | p |
|------------|-------------|---|
|1|3|1|
|2|3|2|
|3|5|1|
|4|5|2|


Once you have done that, test each of your models using your binary classification copy of partition 2 (this should not have been balanced), then calculate and print the TSS score for each result. **NOTE: The model now takes less time to evaluate!**

---

In [ ]:
info_dict = {
    1: [3,1],
    2: [3,2],
    3: [5,1],
    4: [5,2]
}
for key in info_dict.keys():
    neighbor = KNeighborsClassifier(n_neighbors=info_dict.get(key)[0], p=info_dict.get(key)[1])
    neighbor.fit(abt_binary_cpy.drop('lab', axis=1),abt_binary_cpy['lab'])
    score = calc_tss(abt2_binary_cpy['lab'], neighbor.predict(abt2_binary_cpy.drop('lab', axis=1)))
    print(f'Model {key} TSS score: {score}')

---
### Q8

After evaluating the various results from Q7, you will notice that some of the results are improved over the same experiments we conducted in Q2. Additionally, you should also notice a improvement in the speed at which the results were obtained. The question now is will we continue to see these improvements for all of our experiments? So, let's move on and see.

For this question, you will repeat the experiments from Q3, but using the balanced binary classification dataset constructed in Q6. You will still be using the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), as in Q7, but you will again be changing from using the `MinkowskiDistance` metric with different values of `p` to using the `ChebyshevDistance` metric. You will construct two models by changing the number neighbors to 3 and 5.

Once you have done that, test each of your models using your binary classification copy of partition 2 (this should not have been balanced), then calculate and print the TSS score for each result. 

---

In [ ]:
neighbor5 = KNeighborsClassifier(n_neighbors=3, metric='chebyshev')
neighbor5.fit(abt_binary_cpy.drop('lab', axis=1),abt_binary_cpy['lab'])
score5 = calc_tss(abt2_binary_cpy['lab'], neighbor5.predict(abt2_binary_cpy.drop('lab', axis=1)))
print(f'chebyshev 1 TSS score: {score5}')


neighbor6 = KNeighborsClassifier(n_neighbors=5, metric='chebyshev')
neighbor6.fit(abt_binary_cpy.drop('lab', axis=1),abt_binary_cpy['lab'])
score6 = calc_tss(abt2_binary_cpy['lab'], neighbor6.predict(abt2_binary_cpy.drop('lab', axis=1)))
print(f'chebyshev 2 TSS score: {score6}')

---
### Q9

After evaluating the results of Q8 things are looking a little less encouraging, since neither of those results are better than the original dataset. However, the results from Q3 weren't really any better than Q2 in the first place, so not all is lost.  Let's continue on and see how things turn out with models like we used in Q4 since those were actaully an improvement over Q2 originally.

So, in this question, you will again utilize the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), and try several different settings to see how/if using different settings will improve our score. The difference will again be that your are using the balanced binary classification dataset constructed in Q6 to train 8 different instances with the following settings. **(see documentation to know what these are)**

|Model Number| criterion | max_depth | splitter |
|------------|---------|-------------|---|
|1|gini|5|best|
|2|gini|5|random|
|3|gini|None|best|
|4|gini|None|random|
|5|entropy|5|best|
|6|entropy|5|random|
|7|entropy|None|best|
|8|entropy|None|random|



Once you have done that, test each of your models using your binary classification copy of partition 2 (this should not have been balanced), then calculate and print the TSS score for each result. 

---

In [ ]:
info_dict = {
    1:	['gini', 5,	'best'],
    2:	['gini', 5, 'random'],
    3:	['gini', None, 'best'],
    4:	['gini', None, 'random'],
    5:	['entropy', 5, 'best'],
    6:	['entropy', 5, 'random'],
    7:	['entropy', None, 'best'],
    8:	['entropy', None, 'random']
}
for key in info_dict.keys():
    clf = DecisionTreeClassifier(criterion=info_dict.get(key)[0], splitter=info_dict.get(key)[2], max_depth=info_dict.get(key)[1])
    clf.fit(abt_binary_cpy.drop('lab', axis=1),abt_binary_cpy['lab'])
    score = calc_tss(abt2_binary_cpy['lab'], clf.predict(abt2_binary_cpy.drop('lab', axis=1)))
    print(f'Model {key} TTS score: {score}')

---
### Q10

Unlike with [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), it seems that the sampling didn't really help much for the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).  Where before we saw a 3X improvement from one to the other, we now see similar results for both classifiers.  Let's see how this affected our best performing classifier next.

For this question you will again be utilizing the [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) classifier as you did in Q5 but using your balanced binary classification dataset constructed in Q6 to train just 1 model. Once you have done that, test the model using your binary classification copy of partition 2 (this should not have been balanced), then calculate and print the TSS score. 

---

In [ ]:
clf = GaussianNB()
clf.fit(abt_binary_cpy.drop('lab', axis=1),abt_binary_cpy['lab'])
score5 = calc_tss(abt2_binary_cpy['lab'], clf.predict(abt2_binary_cpy.drop('lab', axis=1)))
print(f'GaussianNB TSS score: {score5}')